In [1]:
import math
import numpy as np
import tensorflow as tf
import PIL
from pathlib import Path
import matplotlib.pyplot as plt

random_state = 42

In [3]:
print(tf.__version__)
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

2.0.0
Num GPUs Available:  1


In [ ]:
data_path = Path("F:\Workspace\shrec19retrieval\data")
do_train = True

In [ ]:
class ImageSequenceGenerator(tf.keras.preprocessing.image.Iterator):
    """ Generate image time sequences from directory
        Based on keras DirectoryIterator
        Assuming class labels are integers starting from 0
    """
    def __init__(self, 
                 directory,
                 target_size=(224, 224),
                 batch_size=32,
                 shuffle=True,
                 seed=None):
        self.directory = Path(directory)
        # Count number of classes
        self.classes = [subdir.parts[-1] for subdir in self.directory.iterdir() if subdir.is_dir()]
        self.num_classes = len(classes)
        # Build an index of the image sequences
        # in the different class subfolders.
        
        self.dirnames = []
        self.classes = []
        for classdir in (self.directory / subdir for subdir in self.classes):
            for sampledir in classdir.iterdir():
                if sampledir.is_dir():
                    self.dirnames.append(sampledir)
                    self.classes.append(int(classdir.parts[-1]))
        self.samples = len(self.dirnames)
        self.classes = np.array(self.classes, dtype='int32')
        
        print('Found %d sequences belonging to %d classes.' %
              (self.samples, self.num_classes))